# ntc_rosetta_parse_config

The task ``ntc_rosetta_parse_config`` parses native configuration using
[ntc_rosetta](<https://ntc-rosetta.readthedocs.io/).

The user can specify the configuration to parse by passing the ``config`` parameter. If not
specified, the task will use napalm to retrieve it.

## Requirements

Before jumping into the examples let's import everything we need and create the nornir object:

In [1]:
import json # we only use this to format the output

from nornir import InitNornir
from nornir.core.task import Result, Task
from nornir.plugins.tasks.networking import ntc_rosetta_parse_config

nr = InitNornir(
    inventory={
        "options": {
            "hosts": {
                "rtr00": {
                    "hostname": "localhost",
                    "username": "admin",
                    "password": "admin",
                    "platform": "ios",
                    "connection_options": {
                        "napalm": {
                            "platform": "mock",
                            "extras": {
                                "optional_args": {
                                    "path": "mocked/rtr00/",
                                }
                            }
                        }
                    }
                },
                "rtr01": {
                    "hostname": "localhost",
                    "username": "admin",
                    "password": "admin",
                    "platform": "junos",
                    "connection_options": {
                        "napalm": {
                            "platform": "mock",
                            "extras": {
                                "optional_args": {
                                    "path": "mocked/rtr01/",
                                }
                            }
                        }
                    }
                }
            }
        }
    }
)

## Parsing native configuration using openconfig models

Now let's see how we can leverage the task to retrieve the configuration from the device and return openconfig models:

In [2]:
res = nr.run(
    task=ntc_rosetta_parse_config,
    model="openconfig",
    target="running",
)

In [3]:
# let's print the IOS device
print(json.dumps(res["rtr00"][0].result.raw_value(), indent=4))

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "FastEthernet1",
                "config": {
                    "name": "FastEthernet1",
                    "type": "iana-if-type:ethernetCsmacd",
                    "description": "This is Fa1",
                    "enabled": false
                },
                "subinterfaces": {
                    "subinterface": [
                        {
                            "index": 1,
                            "config": {
                                "index": 1,
                                "description": "This is Fa1.1"
                            }
                        },
                        {
                            "index": 2,
                            "config": {
                                "index": 2,
                                "description": "This is Fa1.2"
                            }
                        }
                    ]
      

In [4]:
# let's print the junos device
print(json.dumps(res["rtr01"][0].result.raw_value(), indent=4))

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "ge-0/0/0",
                "config": {
                    "name": "ge-0/0/0",
                    "type": "iana-if-type:ethernetCsmacd",
                    "enabled": true
                },
                "subinterfaces": {
                    "subinterface": [
                        {
                            "index": 0,
                            "config": {
                                "index": 0,
                                "description": "R1@ge-0/0/0.0->R2-ge-0/0/0.0"
                            }
                        }
                    ]
                }
            },
            {
                "name": "ge-0/0/1",
                "config": {
                    "name": "ge-0/0/1",
                    "type": "iana-if-type:ethernetCsmacd",
                    "enabled": true
                },
                "subinterfaces": {
                    "s

### Filtering down the results

The task supports everything that `ntc_rosetta` does, one of such parameters allows you to narrow down the parsing. Let's do the same as before but getting only vlan configuration:

In [5]:
res = nr.run(
    task=ntc_rosetta_parse_config,
    model="openconfig",
    target="running",
    include=[
        "/openconfig-network-instance:network-instances/network-instance/name",
        "/openconfig-network-instance:network-instances/network-instance/config",
        "/openconfig-network-instance:network-instances/network-instance/vlans",        
    ],
)

In [6]:
# let's print the ios device
print(json.dumps(res["rtr00"][0].result.raw_value(), indent=4))

{
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "name": "default",
                "config": {
                    "name": "default"
                },
                "vlans": {
                    "vlan": [
                        {
                            "vlan-id": 10,
                            "config": {
                                "vlan-id": 10,
                                "name": "prod",
                                "status": "ACTIVE"
                            }
                        },
                        {
                            "vlan-id": 20,
                            "config": {
                                "vlan-id": 20,
                                "name": "dev",
                                "status": "SUSPENDED"
                            }
                        }
                    ]
                }
            }
        ]
    }
}


In [7]:
# let's print the junos device, which has no vlans
print(json.dumps(res["rtr01"][0].result.raw_value(), indent=4))

{
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "name": "default",
                "config": {
                    "name": "default"
                }
            }
        ]
    }
}


## Getting other models

The task supports everything that `ntc_rosetta`, that means you can also retrieve other models, as long as it's supported by `ntc_rosetta`. Let's try with the `ntc_yang_models`:

In [8]:
res = nr.run(
    task=ntc_rosetta_parse_config,
    model="ntc",
    target="running",
    include=[
        "/ntc-vlan:vlan/config/vlans",
    ],        
)

In [9]:
# let's print the ios device
print(json.dumps(res["rtr00"][0].result.raw_value(), indent=4))

{
    "ntc-vlan:vlan": {
        "config": {
            "vlans": [
                {
                    "vlan-id": 10,
                    "name": "prod",
                    "active": true
                },
                {
                    "vlan-id": 20,
                    "name": "dev",
                    "active": false
                }
            ]
        }
    }
}


In [10]:
# let's print the junos device, which has no vlans
print(json.dumps(res["rtr01"][0].result.raw_value(), indent=4))

{}
